In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/3rd_place_combination

res0: Boolean = true

In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/round_16_teams

res1: Boolean = true

In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/round_16_matches

res2: Boolean = true

In [0]:
from pyspark.sql.functions import rand,when,row_number
from pyspark.sql import SparkSession
from pyspark.sql.window import Window

spark = SparkSession.builder \
    .appName("Euro_2024") \
    .getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:

teams_all = [
        # Group A
    (1, "Germany", "A"),
    (2, "Hungary", "A"),
    (3, "Scotland", "A"),
    (4, "Switzerland", "A"),

    # Group B
    (5, "Albania", "B"),
    (6, "Croatia", "B"),
    (7, "Italy", "B"),
    (8, "Spain", "B"),

    # Group C
    (9, "Denmark", "C"),
    (10, "England", "C"),
    (11, "Slovenia", "C"),
    (12, "Serbia", "C"),

    # Group D
    (13, "Austria", "D"),
    (14, "France", "D"),
    (15, "Netherlands", "D"),
    (16, "Playoff Winner 1", "D"),

    # Group E
    (17, "Belgium", "E"),
    (18, "Romania", "E"),
    (19, "Slovakia", "E"),
    (20, "Playoff Winner 2", "E"),

    # Group F
    (21, "Czech Republic", "F"),
    (22, "Portugal", "F"),
    (23, "Turkey", "F"),
    (24, "Playoff Winner 3", "F")
]

    

In [0]:
teams_columns = ["TeamID", "TeamName", "TeamGroup"]
teams_df = spark.createDataFrame(teams_all, teams_columns)
teams_df.createOrReplaceTempView("teams")

In [0]:
display(spark.sql("SELECT * FROM teams"))

TeamID,TeamName,TeamGroup
1,Germany,A
2,Hungary,A
3,Scotland,A
4,Switzerland,A
5,Albania,B
6,Croatia,B
7,Italy,B
8,Spain,B
9,Denmark,C
10,England,C


In [0]:
gs_matches_df = spark.sql("""
    SELECT  t1.TeamName AS Team_X, t2.TeamName AS Team_Y, t1.TeamGroup AS Team_X_Group, t2.TeamGroup AS Team_Y_Group
    FROM teams t1
    JOIN teams t2 ON t1.TeamGroup = t2.TeamGroup AND t1.TeamID < t2.TeamID  /*(TeamA, TeamB) != (TeamB, TeamA)*/
    """)
display(gs_matches_df)

Team_X,Team_Y,Team_X_Group,Team_Y_Group
Germany,Hungary,A,A
Germany,Scotland,A,A
Germany,Switzerland,A,A
Hungary,Scotland,A,A
Hungary,Switzerland,A,A
Scotland,Switzerland,A,A
Albania,Croatia,B,B
Albania,Italy,B,B
Albania,Spain,B,B
Croatia,Italy,B,B


In [0]:
gs_matches_df = gs_matches_df.withColumn("X_TeamGoals", (rand() * 6).cast("int")) 
gs_matches_df = gs_matches_df.withColumn("Y_TeamGoals", (rand() * 6).cast("int")) 


gs_matches_df.createOrReplaceTempView("group_stage_goals")

In [0]:
display(gs_matches_df)

Team_X,Team_Y,Team_X_Group,Team_Y_Group,X_TeamGoals,Y_TeamGoals
Germany,Hungary,A,A,0,3
Germany,Scotland,A,A,3,3
Germany,Switzerland,A,A,0,3
Hungary,Scotland,A,A,1,1
Hungary,Switzerland,A,A,3,3
Scotland,Switzerland,A,A,1,2
Albania,Croatia,B,B,1,5
Albania,Italy,B,B,5,2
Albania,Spain,B,B,1,3
Croatia,Italy,B,B,3,5


In [0]:
# Calculate additional match statistics
gs_match_stats_df = spark.sql("""
    SELECT 
        TeamName AS Team,
        TeamGroup,
        COUNT(*) AS Pld,
        SUM(CASE WHEN X_TeamGoals = Y_TeamGoals THEN 1 ELSE 0 END) AS D,
        SUM(CASE WHEN X_TeamGoals > Y_TeamGoals THEN 1 ELSE 0 END) AS W,
        SUM(CASE WHEN X_TeamGoals < Y_TeamGoals THEN 1 ELSE 0 END) AS L,
        SUM(X_TeamGoals) AS GF,
        SUM(Y_TeamGoals) AS GA,
        SUM(X_TeamGoals) - SUM(Y_TeamGoals) AS GD,
        SUM(
            CASE 
                WHEN X_TeamGoals > Y_TeamGoals THEN 3
                WHEN X_TeamGoals = Y_TeamGoals THEN 1
                ELSE 0 
            END
        ) AS Pts
      
    FROM (
        SELECT Team_X AS TeamName, X_TeamGoals, Y_TeamGoals, Team_X_Group AS TeamGroup FROM group_stage_goals
        UNION ALL
        SELECT Team_Y AS TeamName, Y_TeamGoals, X_TeamGoals, Team_Y_Group AS TeamGroup FROM group_stage_goals
    )
    GROUP BY TeamName, TeamGroup
    ORDER BY Pts DESC, GD DESC
    """)

In [0]:
display(gs_match_stats_df)

Team,TeamGroup,Pld,D,W,L,GF,GA,GD,Pts
Serbia,C,3,0,3,0,10,0,10,9
Netherlands,D,3,1,2,0,13,7,6,7
Portugal,F,3,1,2,0,7,3,4,7
Switzerland,A,3,1,2,0,8,4,4,7
Czech Republic,F,3,0,2,1,12,7,5,6
Spain,B,3,0,2,1,9,7,2,6
Italy,B,3,0,2,1,12,11,1,6
France,D,3,2,1,0,12,7,5,5
Hungary,A,3,2,1,0,7,4,3,5
Playoff Winner 2,E,3,2,1,0,6,5,1,5


In [0]:
# Create a window specification to partition by Group and order by Points and Goal Difference
windowSpec = Window.partitionBy(gs_match_stats_df["TeamGroup"]).orderBy(gs_match_stats_df["Pts"].desc(), gs_match_stats_df["GD"].desc())

# Assign positions within each group
gs_match_stats_df = gs_match_stats_df.withColumn("Pos", row_number().over(windowSpec))
display(gs_match_stats_df)

Team,TeamGroup,Pld,D,W,L,GF,GA,GD,Pts,Pos
Switzerland,A,3,1,2,0,8,4,4,7,1
Hungary,A,3,2,1,0,7,4,3,5,2
Scotland,A,3,2,0,1,5,6,-1,2,3
Germany,A,3,1,0,2,3,9,-6,1,4
Spain,B,3,0,2,1,9,7,2,6,1
Italy,B,3,0,2,1,12,11,1,6,2
Croatia,B,3,0,1,2,9,9,0,3,3
Albania,B,3,0,1,2,7,10,-3,3,4
Serbia,C,3,0,3,0,10,0,10,9,1
England,C,3,1,1,1,7,6,1,4,2


In [0]:
gs_match_stats_df.createOrReplaceTempView("gs_stats_table");

In [0]:
# %sql
# DROP TABLE IF EXISTS 3rd_place_Combinations;

In [0]:
# %fs
# rm -r dbfs:/user/hive/warehouse/3rd_place_combinations/part-00000-46a51ba8-3585-4b5a-a2f7-16ffa6c7b13c-c000.snappy.parquet

In [0]:
%sql
CREATE TABLE IF NOT EXISTS 3rd_place_Combination (
  ID INTEGER,
  GroupId VARCHAR(4),
  Team1B CHAR(1),
  Team1C CHAR(1),
  Team1E CHAR(1),
  Team1F CHAR(1)
)
-- LOCATION 'dbfs:/user/hive/warehouse/3rdd_place_Combination';

In [0]:
%sql
INSERT INTO 3rd_place_Combination (ID, GroupId, Team1B, Team1C, Team1E, Team1F) VALUES
(1, 'ABCD', 'A', 'D', 'B', 'C'),
(2, 'ABCE', 'A', 'E', 'B', 'C'),
(3, 'ABCF', 'A', 'F', 'B', 'C'),
(4, 'ABDE', 'D', 'E', 'A', 'B'),
(5, 'ABDF', 'D', 'F', 'A', 'B'),
(6, 'ABEF', 'E', 'F', 'B', 'A'),
(7, 'ACDE', 'E', 'D', 'C', 'A'),
(8, 'ACDF', 'F', 'D', 'C', 'A'),
(9, 'ACEF', 'E', 'F', 'C', 'A'),
(10, 'ADEF', 'E', 'F', 'D', 'A'),
(11, 'BCDE', 'E', 'D', 'B', 'C'),
(12, 'BCDF', 'F', 'D', 'C', 'B'),
(13, 'BCEF', 'F', 'E', 'C', 'B'),
(14, 'BDEF', 'F', 'E', 'D', 'B'),
(15, 'CDEF', 'F', 'E', 'D', 'C');

num_affected_rows,num_inserted_rows
15,15


In [0]:
%sql
select * from 3rd_place_Combination

ID,GroupId,Team1B,Team1C,Team1E,Team1F
1,ABCD,A,D,B,C
2,ABCE,A,E,B,C
3,ABCF,A,F,B,C
4,ABDE,D,E,A,B
5,ABDF,D,F,A,B
6,ABEF,E,F,B,A
7,ACDE,E,D,C,A
8,ACDF,F,D,C,A
9,ACEF,E,F,C,A
10,ADEF,E,F,D,A


In [0]:
%sql
WITH thirds AS (
    SELECT 
        TeamGroup,
        ROW_NUMBER() OVER (ORDER BY Pts, GD) AS rank 
    FROM 
        gs_stats_table 
    WHERE 
        pos = 3 
    LIMIT 4
),
-- select * from thirds;
GrpConcat AS (
    SELECT 
         CONCAT_WS('', sort_array(COLLECT_LIST(TeamGroup))) AS grp
    FROM 
        thirds 
),
-- select * from GrpConcat ;
combo AS (
    SELECT * 
    FROM 3rd_place_Combination 
    WHERE GroupId = (SELECT grp FROM GrpConcat)
)
select * from combo ;
-- SELECT Team1B FROM combo;

ID,GroupId,Team1B,Team1C,Team1E,Team1F
8,ACDF,F,D,C,A


In [0]:

%sql
CREATE or replace TABLE round_16_teams(
  M_ID INT,
  Team_1 VARCHAR(25),
  Team_2 VARCHAR(25)
);


In [0]:
%sql
WITH thirds AS (
    SELECT 
        TeamGroup,
        ROW_NUMBER() OVER (ORDER BY Pts, GD) AS rank 
    FROM 
        gs_stats_table 
    WHERE 
        pos = 3 
    LIMIT 4
),
-- select * from thirds;
GrpConcat AS (
    SELECT 
         CONCAT_WS('', sort_array(COLLECT_LIST(TeamGroup))) AS grp
    FROM 
        thirds 
),
-- select * from GrpConcat ;
combo AS (
    SELECT * 
    FROM 3rd_place_Combination 
    WHERE GroupId = (SELECT grp FROM GrpConcat)
)
-- select * from combo ;
-- SELECT Team1B FROM combo;
INSERT INTO round_16_teams(
    SELECT 1 AS M_ID,
        (SELECT Team FROM gs_stats_table WHERE Pos = 1 AND TeamGroup = 'B') as Team1B,
        (SELECT Team FROM gs_stats_table WHERE Pos = 3 AND TeamGroup = (SELECT Team1B FROM combo)) as Team3rd_1
    UNION ALL
    SELECT 2 as M_ID,
        (SELECT Team FROM gs_stats_table WHERE Pos = 1 AND TeamGroup = 'A') as Team1A,
        (SELECT Team FROM gs_stats_table WHERE Pos = 2 AND TeamGroup = 'C') as Team2C
    UNION ALL
    SELECT 3 AS M_ID,
        (SELECT Team FROM gs_stats_table WHERE Pos = 1 AND TeamGroup = 'F') as Team1F,
        (SELECT Team FROM gs_stats_table WHERE Pos = 3 AND TeamGroup = (SELECT Team1F FROM combo)) as Team3rd_2
    UNION ALL
    SELECT 4 as M_ID,
        (SELECT Team FROM gs_stats_table WHERE Pos = 2 AND TeamGroup = 'D') as Team1D,
        (SELECT Team FROM gs_stats_table WHERE Pos = 2 AND TeamGroup = 'E') as Team2E
    UNION ALL
    SELECT 5 AS M_ID,
        (SELECT Team FROM gs_stats_table WHERE Pos = 1 AND TeamGroup = 'E') as Team1E,
        (SELECT Team FROM gs_stats_table WHERE Pos = 3 AND TeamGroup = (SELECT Team1E FROM combo)) as Team3rd_3
    UNION ALL
    SELECT 6 as M_ID,
        (SELECT Team FROM gs_stats_table WHERE Pos = 1 AND TeamGroup = 'D') as Team1D,
        (SELECT Team FROM gs_stats_table WHERE Pos = 2 AND TeamGroup = 'F') as Team2F
    UNION ALL
    SELECT 7 AS M_ID,
        (SELECT Team FROM gs_stats_table WHERE Pos = 1 AND TeamGroup = 'C') as Team1C,
        (SELECT Team FROM gs_stats_table WHERE Pos = 3 AND TeamGroup = (SELECT Team1C FROM combo)) as Team3rd_4
    UNION ALL
    SELECT 8 as M_ID,
        (SELECT Team FROM gs_stats_table WHERE Pos = 2 AND TeamGroup = 'A') as Team2A,
        (SELECT Team FROM gs_stats_table WHERE Pos = 2 AND TeamGroup = 'B') as Team2B
    FROM combo
);

num_affected_rows,num_inserted_rows
8,8


In [0]:
%sql
select * from round_16_teams order by M_ID;

M_ID,Team_1,Team_2
1,Spain,Turkey
2,Switzerland,England
3,Portugal,Scotland
4,France,Slovakia
5,Playoff Winner 2,Denmark
6,Netherlands,Czech Republic
7,Serbia,Austria
8,Hungary,Italy


In [0]:
%sql
CREATE TABLE round_16_matches (
    Match STRING,
    Team_1 VARCHAR(50),
    Team_2 VARCHAR(50),
    Team_1_Goals INT,
    Team_2_Goals INT,
    Winner VARCHAR(50)
);

In [0]:
%sql
INSERT INTO round_16_matches (
    SELECT
        "RS " || ROW_NUMBER() OVER (ORDER BY M_ID) AS Match,
        Team_1 AS Team_1,
        Team_2 AS Team_2,
        FLOOR(rand() * 5) AS Team_1_Goals,
        FLOOR(rand() * 5) AS Team_2_Goals,
        CASE
            WHEN Team_1_Goals > Team_2_Goals THEN Team_1
            WHEN Team_2_Goals > Team_1_Goals THEN Team_2
            ELSE CASE WHEN rand() > 0.5 THEN Team_1 ELSE Team_2 END
        END AS Winner
    FROM round_16_teams
);
SELECT * FROM round_16_matches;

Match,Team_1,Team_2,Team_1_Goals,Team_2_Goals,Winner
RS 1,Spain,Turkey,0,2,Turkey
RS 2,Switzerland,England,3,3,Switzerland
RS 3,Portugal,Scotland,2,2,Scotland
RS 4,France,Slovakia,4,0,France
RS 5,Playoff Winner 2,Denmark,3,3,Playoff Winner 2
RS 6,Netherlands,Czech Republic,1,1,Czech Republic
RS 7,Serbia,Austria,1,2,Austria
RS 8,Hungary,Italy,0,4,Italy
3rd Place,null,null,5,4,null
Final,null,null,3,3,null


In [0]:
# %sql
# CREATE or replace TABLE QF_teams(
#   M_ID INT,
#   Team_1 VARCHAR(25),
#   Team_2 VARCHAR(25)
# );

In [0]:
# %sql
# INSERT INTO QF_teams (
#         SELECT 1 AS M_ID, 
#                (SELECT Winner FROM round_16_matches WHERE Match = 'RS 1') AS Team_1,
#                (SELECT Winner FROM round_16_matches WHERE Match = 'RS 2') AS Team_2
#         UNION ALL
#         SELECT 2 AS M_ID,  
#                (SELECT Winner FROM round_16_matches WHERE Match = 'RS 3') AS Team_1,
#                (SELECT Winner FROM round_16_matches WHERE Match = 'RS 4') AS Team_2
#         UNION ALL
#         SELECT 3 AS M_ID,  
#                (SELECT Winner FROM round_16_matches WHERE Match = 'RS 5') AS Team_1,
#                (SELECT Winner FROM round_16_matches WHERE Match = 'RS 6') AS Team_2
#         UNION ALL
#         SELECT 4 AS M_ID, 
#                (SELECT Winner FROM round_16_matches WHERE Match = 'RS 7') AS Team_1,
#                (SELECT Winner FROM round_16_matches WHERE Match = 'RS 8') AS Team_2 
#         FROM round_16_matches
# );

In [0]:
# %sql
# select * from QF_teams;

In [0]:
%sql
INSERT INTO round_16_matches (
    WITH QuarterFinal AS (
        SELECT 'QF 1' AS QFMatch, 
               (SELECT Winner FROM round_16_matches WHERE Match = 'RS 1') AS Team_1,
               (SELECT Winner FROM round_16_matches WHERE Match = 'RS 2') AS Team_2
        UNION ALL
        SELECT 'QF 2' AS QFMatch, 
               (SELECT Winner FROM round_16_matches WHERE Match = 'RS 3') AS Team_1,
               (SELECT Winner FROM round_16_matches WHERE Match = 'RS 4') AS Team_2
        UNION ALL
        SELECT 'QF 3' AS QFMatch, 
               (SELECT Winner FROM round_16_matches WHERE Match = 'RS 5') AS Team_1,
               (SELECT Winner FROM round_16_matches WHERE Match = 'RS 6') AS Team_2
        UNION ALL
        SELECT 'QF 4' AS QFMatch, 
               (SELECT Winner FROM round_16_matches WHERE Match = 'RS 7') AS Team_1,
               (SELECT Winner FROM round_16_matches WHERE Match = 'RS 8') AS Team_2
    )
    SELECT
        QFMatch,
        Team_1 AS Team_1_Name,
        Team_2 AS Team_2_Name,
        ROUND(RAND() * 5) AS Team_1_Goals,
        ROUND(RAND() * 5) AS Team_2_Goals,
        CASE
            WHEN Team_1_Goals > Team_2_Goals THEN Team_1
            WHEN Team_2_Goals > Team_1_Goals THEN Team_2
            ELSE CASE WHEN rand() > 0.5 THEN Team_1 ELSE Team_2 END
        END AS Winner
    FROM QuarterFinal
);


num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
select * from round_16_matches;

Match,Team_1,Team_2,Team_1_Goals,Team_2_Goals,Winner
RS 1,Spain,Turkey,0,2,Turkey
RS 2,Switzerland,England,3,3,Switzerland
RS 3,Portugal,Scotland,2,2,Scotland
RS 4,France,Slovakia,4,0,France
RS 5,Playoff Winner 2,Denmark,3,3,Playoff Winner 2
RS 6,Netherlands,Czech Republic,1,1,Czech Republic
RS 7,Serbia,Austria,1,2,Austria
RS 8,Hungary,Italy,0,4,Italy
QF 3,Playoff Winner 2,Czech Republic,4,2,Playoff Winner 2
QF 1,Turkey,Switzerland,2,4,Switzerland


In [0]:
%sql
INSERT INTO round_16_matches (
    WITH SemiFinal AS (
        SELECT 'SF 1' AS SFMatch, 
               (SELECT Winner FROM round_16_matches WHERE Match = 'QF 1') AS Team_1,
               (SELECT Winner FROM round_16_matches WHERE Match = 'QF 2') AS Team_2
        UNION ALL
        SELECT 'SF 2' AS SFMatch, 
               (SELECT Winner FROM round_16_matches WHERE Match = 'QF 3') AS Team_1,
               (SELECT Winner FROM round_16_matches WHERE Match = 'QF 4') AS Team_2
    )
    SELECT
        SFMatch,
        Team_1 AS Team_1_Name,
        Team_2 AS Team_2_Name,
        ROUND(RAND() * 5) AS Team_1_Goals,
        ROUND(RAND() * 5) AS Team_2_Goals,
        CASE
            WHEN Team_1_Goals > Team_2_Goals THEN Team_1
            WHEN Team_2_Goals > Team_1_Goals THEN Team_2
            ELSE CASE WHEN rand() > 0.5 THEN Team_1 ELSE Team_2 END
        END AS Winner
    FROM SemiFinal
);

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-1142063495687293>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-1142063495687293>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("SU5TRVJUIElOVE8gcm91bmRfMTZfbWF0Y2hlcyAoCiAgICBXSVRIIFNlbWlGaW5hbCBBUyAoCiAgICAgICAgU0VMRUNUICdTRiAxJyBBUyBTRk1hdGNoLCAKICAgICAgICAgICAgICAgKFNFTEVDVCBXaW5uZXIgRlJPTSByb3VuZF8xNl9tYXRjaGVzIFdIRVJFIE1hdGNoID0gJ1FGIDEnKSBBUyBUZWFtXzEsCiAgICAgICAgICAgICAgIChTRUxFQ1QgV2lubmVyIEZST00gcm91bmRfMTZfbWF0Y2hlcyBXSEVSRSBNYXRjaCA9ICdRRiAyJykgQVMgVGVhbV8yCiAgICAgICAgVU5JT04gQUxMCiAgICAgICAgU0VMRUNUICdTRiAyJyBBUyBTRk1hdGNoLCAKICAgICAgICAgICAgICAgKFNFTEVDVCBXaW5uZXIgRlJPTSByb3VuZF8xNl9t

In [0]:
%sql
select * from round_16_matches;

Match,Team_1,Team_2,Team_1_Goals,Team_2_Goals,Winner
RS 1,Spain,Turkey,0,2,Turkey
RS 2,Switzerland,England,3,3,Switzerland
RS 3,Portugal,Scotland,2,2,Scotland
RS 4,France,Slovakia,4,0,France
RS 5,Playoff Winner 2,Denmark,3,3,Playoff Winner 2
RS 6,Netherlands,Czech Republic,1,1,Czech Republic
RS 7,Serbia,Austria,1,2,Austria
RS 8,Hungary,Italy,0,4,Italy
QF 3,Playoff Winner 2,Czech Republic,4,2,Playoff Winner 2
QF 1,Turkey,Switzerland,2,4,Switzerland


In [0]:
%sql
INSERT INTO round_16_matches (
    WITH SemiFinalLoss AS (
        SELECT 'SF 1' AS SFMatch, 
               (SELECT CASE WHEN Winner = Team_1 THEN Team_2 ELSE Team_1 END FROM round_16_matches WHERE Match = 'SF 1') AS Loss_1,
               (SELECT CASE WHEN Winner = Team_1 THEN Team_2 ELSE Team_1 END FROM round_16_matches WHERE Match = 'SF 2') AS Loss_2
    ),
    ThirdPlace AS (
        SELECT
            '3rd Place' AS ThirdPlaceMatch,
            Loss_1 AS Team_1_Name,
            Loss_2 AS Team_2_Name,
            ROUND(RAND() * 5) AS Team_1_Goals,
            ROUND(RAND() * 5) AS Team_2_Goals,
            CASE
                WHEN ROUND(RAND()) = 1 THEN Loss_1
            WHEN Team_1_Goals > Team_2_Goals THEN Loss_1
            WHEN Team_2_Goals > Team_1_Goals THEN Loss_2
            ELSE CASE WHEN rand() > 0.5 THEN Loss_1 ELSE Loss_2 END
            END AS Winner
        FROM SemiFinalLoss
    )
    SELECT * FROM ThirdPlace
);


num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
select * from round_16_matches;

Match,Team_1,Team_2,Team_1_Goals,Team_2_Goals,Winner
RS 1,Spain,Turkey,0,2,Turkey
RS 2,Switzerland,England,3,3,Switzerland
RS 3,Portugal,Scotland,2,2,Scotland
RS 4,France,Slovakia,4,0,France
RS 5,Playoff Winner 2,Denmark,3,3,Playoff Winner 2
RS 6,Netherlands,Czech Republic,1,1,Czech Republic
RS 7,Serbia,Austria,1,2,Austria
RS 8,Hungary,Italy,0,4,Italy
QF 3,Playoff Winner 2,Czech Republic,4,2,Playoff Winner 2
QF 1,Turkey,Switzerland,2,4,Switzerland


In [0]:
%sql
INSERT INTO round_16_matches (
    WITH FinalMatch AS (
        SELECT 'Final' AS FinalMatch,
               (SELECT Winner FROM round_16_matches WHERE Match = 'SF 1') AS Team_1,
               (SELECT Winner FROM round_16_matches WHERE Match = 'SF 2') AS Team_2
    )
    SELECT
        FinalMatch,
        Team_1 AS Team_1_Name,
        Team_2 AS Team_2_Name,
        ROUND(RAND() * 5) AS Team_1_Goals,
        ROUND(RAND() * 5) AS Team_2_Goals,
        CASE
            WHEN Team_1_Goals > Team_2_Goals THEN Team_1
            WHEN Team_2_Goals > Team_1_Goals THEN Team_2
            ELSE CASE WHEN rand() > 0.5 THEN Team_1 ELSE Team_2 END
        END AS Winner
    FROM FinalMatch
);

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
select * from round_16_matches;

Match,Team_1,Team_2,Team_1_Goals,Team_2_Goals,Winner
RS 1,Spain,Turkey,0,2,Turkey
RS 2,Switzerland,England,3,3,Switzerland
RS 3,Portugal,Scotland,2,2,Scotland
RS 4,France,Slovakia,4,0,France
RS 5,Playoff Winner 2,Denmark,3,3,Playoff Winner 2
RS 6,Netherlands,Czech Republic,1,1,Czech Republic
RS 7,Serbia,Austria,1,2,Austria
RS 8,Hungary,Italy,0,4,Italy
QF 3,Playoff Winner 2,Czech Republic,4,2,Playoff Winner 2
QF 1,Turkey,Switzerland,2,4,Switzerland


In [0]:
%sql
WITH MatchDetails AS (
    SELECT 
      ROW_NUMBER() OVER (ORDER BY Match) AS RowNum,
    CASE 
        WHEN Match IN ('RS 1', 'RS 5') THEN 1
        WHEN Match IN ('QF 1', 'QF 3') THEN 2
        WHEN Match IN ('RS 2', 'RS 6') THEN 3
        WHEN Match IN ('RS 3', 'RS 7') THEN 4
        WHEN Match IN ('QF 2', 'QF 4') THEN 5
        WHEN Match IN ('RS 4', 'RS 8') THEN 6
        WHEN Match = 'SF 1' THEN 6
        WHEN Match = 'SF 2' THEN 6
        WHEN Match = 'Final' THEN 6
        WHEN Match = '3rd Place' THEN 8
    END AS ColumnNum,
        

        CASE 
            WHEN Match = 'RS 1' THEN CONCAT('RS 1: ', Team_1, ' vs ', Team_2, ', ', Team_1_Goals, '-', Team_2_Goals, ', Winner: ', Winner)
            WHEN Match = 'RS 2' THEN CONCAT('RS 2: ', Team_1, ' vs ', Team_2, ', ', Team_1_Goals, '-', Team_2_Goals, ', Winner: ', Winner)
            WHEN Match = 'RS 3' THEN CONCAT('RS 3: ', Team_1, ' vs ', Team_2, ', ', Team_1_Goals, '-', Team_2_Goals, ', Winner: ', Winner)
            WHEN Match = 'RS 4' THEN CONCAT('RS 4: ', Team_1, ' vs ', Team_2, ', ', Team_1_Goals, '-', Team_2_Goals, ', Winner: ', Winner)
            ELSE '' 
        END AS R16_A,
        CASE 
            WHEN Match = 'QF 1' THEN CONCAT('QF 1: ', Team_1, ' vs ', Team_2, ', ', Team_1_Goals, '-', Team_2_Goals, ', Winner: ', Winner)
            WHEN Match = 'QF 2' THEN CONCAT('QF 2: ', Team_1, ' vs ', Team_2, ', ', Team_1_Goals, '-', Team_2_Goals, ', Winner: ', Winner)
            ELSE '' 
        END AS QF_A,
        CASE 
            WHEN Match = 'SF 1' THEN CONCAT('SF 1: ', Team_1, ' vs ', Team_2, ', ', Team_1_Goals, '-', Team_2_Goals, ', Winner: ', Winner)
            ELSE '' 
        END AS Semi_Final_A,
        CASE 
            WHEN Match = 'Final' THEN CONCAT('Final: ', Team_1, ' vs ', Team_2, ', ', Team_1_Goals, '-', Team_2_Goals, ', Winner: ', Winner)
            WHEN Match = '3rd Place' THEN CONCAT('3rd Place: ', Team_1, ' vs ', Team_2, ', ', Team_1_Goals, '-', Team_2_Goals, ', Winner: ', Winner)
            ELSE '' 
        END AS Final,
        CASE 
            WHEN Match = 'SF 2' THEN CONCAT('SF 2: ', Team_1, ' vs ', Team_2, ', ', Team_1_Goals, '-', Team_2_Goals, ', Winner: ', Winner)
            ELSE '' 
        END AS Semi_Final_B,
        CASE 
            WHEN Match = 'QF 3' THEN CONCAT('QF 3: ', Team_1, ' vs ', Team_2, ', ', Team_1_Goals, '-', Team_2_Goals, ', Winner: ', Winner)
            WHEN Match = 'QF 4' THEN CONCAT('QF 4: ', Team_1, ' vs ', Team_2, ', ', Team_1_Goals, '-', Team_2_Goals, ', Winner: ', Winner)
            ELSE '' 
        END AS QF_B,
        CASE 
            WHEN Match = 'RS 5' THEN CONCAT('RS 5: ', Team_1, ' vs ', Team_2, ', ', Team_1_Goals, '-', Team_2_Goals, ', Winner: ', Winner)
            WHEN Match = 'RS 6' THEN CONCAT('RS 6: ', Team_1, ' vs ', Team_2, ', ', Team_1_Goals, '-', Team_2_Goals, ', Winner: ', Winner)
            WHEN Match = 'RS 7' THEN CONCAT('RS 7: ', Team_1, ' vs ', Team_2, ', ', Team_1_Goals, '-', Team_2_Goals, ', Winner: ', Winner)
            WHEN Match = 'RS 8' THEN CONCAT('RS 8: ', Team_1, ' vs ', Team_2, ', ', Team_1_Goals, '-', Team_2_Goals, ', Winner: ', Winner)
            ELSE '' 
        END AS R16_B


    FROM round_16_matches
      
)

SELECT 
 

    
    R16_A,
    QF_A,
    Semi_Final_A,
    Final,
    Semi_Final_B,
    QF_B,
    R16_B
FROM 
    MatchDetails
WHERE
    RowNum IS NOT NULL
ORDER BY
    ColumnNum, RowNum;

R16_A,QF_A,Semi_Final_A,Final,Semi_Final_B,QF_B,R16_B
"RS 1: Spain vs Turkey, 0-2, Winner: Turkey",,,,,,
,,,,,,"RS 5: Playoff Winner 2 vs Denmark, 3-3, Winner: Playoff Winner 2"
,"QF 1: Turkey vs Switzerland, 2-4, Winner: Switzerland",,,,,
,null,,,,,
,,,,,"QF 3: Playoff Winner 2 vs Czech Republic, 4-2, Winner: Playoff Winner 2",
,,,,,null,
"RS 2: Switzerland vs England, 3-3, Winner: Switzerland",,,,,,
,,,,,,"RS 6: Netherlands vs Czech Republic, 1-1, Winner: Czech Republic"
"RS 3: Portugal vs Scotland, 2-2, Winner: Scotland",,,,,,
,,,,,,"RS 7: Serbia vs Austria, 1-2, Winner: Austria"
